In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<style>
input {
    background-color: #3c83bb;
    color: white;
    border: none;
    font-family: sans-serif;
    padding: 8px;
    border-radius: 10px;
}
</style>
<form action="javascript:code_toggle()"><input type="submit" value="See Raw Code"></form>''')


In [2]:
import pandas as pd
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive_output
from IPython.display import display, clear_output
from jupyterthemes import jtplot

from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from bokeh.palettes import d3

jtplot.style(theme='chesterish', grid=False)
output_notebook()

Loading BokehJS ...

In [3]:
df_prod_factors = pd.read_csv("../Data/Productivity_Factors.csv", index_col=0)
df_prod_grades_alpha = pd.read_csv("../Data/Productivity_Grades_Alpha.csv", index_col=0)
df_prod_grades_num = pd.read_csv("../Data/Productivity_Grades_Num.csv", index_col=0)
df_prof_factors = pd.read_csv("../Data/Profitability_Factors.csv", index_col=0)
df_prof_grades_alpha = pd.read_csv("../Data/Profitability_Grades_Alpha.csv", index_col=0, dtype={'High GMR': object, 'High Sales and GM Weight': object, 'Equal Weights': object})
df_prof_grades_num = pd.read_csv("../Data/Profitability_Grades_Num.csv", index_col=0)

In [4]:
df_display_prod = df_prod_grades_num[['Sales Office', 'Date (YYYYMM)', 'High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()
df_display_prod[['High Sales Weight Alpha', 'Equal Sales and GM Weight Alpha', 'Equal Weights Alpha']] = df_prod_grades_alpha[['High Sales Weight', 'Equal Sales and GM Weight', 'Equal Weights']].copy()

df_display_prof = df_prof_grades_num[['Sales Office', 'Date (YYYYMM)', 'High GMR', 'High Sales and GM Weight', 'Equal Weights']].copy()
df_display_prof[['High GMR Alpha', 'High Sales and GM Weight Alpha', 'Equal Weights Alpha']] = (df_prof_grades_alpha[['High GMR', 'High Sales and GM Weight', 'Equal Weights']].copy())

df_display_prof.replace("nan", np.nan, inplace=True)

dates = ['201701', '201702', '201703', '201704', '201705', '201706', '201707', 
         '201708', '201709', '201710', '201711', '201712', '201801', '201802', 
         '201803', '201804', '201805', '201806', '201807', '201808', '201809', 
         '201810', '201811', '201812', '201901', '201902', '201903']

In [5]:
Productivity = widgets.Dropdown(
    options=['Equal Weights', 'High Sales Weight', 'Equal Sales and GM Weight'],
    value='Equal Weights',
    description='Productivity',
)

Profitability = widgets.Dropdown(
    options=['Equal Weights', 'High GMR', 'High Sales and GM Weight'],
    value='Equal Weights',
    description='Profitability',
)

timeline = widgets.Dropdown(
    options=dates,
    value='201701',
    description='Timeline',
)

hbox = widgets.HBox([Productivity, Profitability, timeline])

def show_this(Productivity, Profitability, timeline):
    temp = pd.DataFrame()
    
    temp_df_display_prod = df_display_prod[df_display_prod['Date (YYYYMM)']==int(timeline)]
    temp_df_display_prof = df_display_prof[df_display_prof['Date (YYYYMM)']==int(timeline)]
    
    temp['Legend'] = temp_df_display_prod[Productivity+' Alpha'] + temp_df_display_prof[Profitability+' Alpha']

    
    legend = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4', np.nan]
    colors = d3["Category20b"][len(legend)]
    colormap = dict(zip(legend, colors))
    
    temp['Colors'] = [colormap[x] for x in temp.Legend]
    
    source = ColumnDataSource(data = {'Sales_Office' : temp_df_display_prod['Sales Office'], 
                                  'Date_(YYYYMM)': temp_df_display_prod['Date (YYYYMM)'], 
                                  Productivity+'_Prod': temp_df_display_prod[Productivity],
                                  Productivity+'_Alpha_Prod': temp_df_display_prod[Productivity+' Alpha'],
                                  Profitability+'_Prof': temp_df_display_prof[Profitability],
                                  Profitability+'_Alpha_Prof': temp_df_display_prof[Profitability+' Alpha'],
                                  'Legend': temp_df_display_prod[Productivity+' Alpha'] + temp_df_display_prof[Profitability+' Alpha'].map(str),
                                  'Colors': temp['Colors']
                                })
    
    p = figure(plot_height=600, plot_width=750)
    p.xaxis.axis_label = 'Productivity'
    p.yaxis.axis_label = 'Profitability'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.background_fill_color = "#272222"
    
    sp = p.scatter(x= Productivity+'_Prod', y=Profitability+'_Prof', source=source, color = 'Colors', size=6, fill_alpha=0.50)
    
    p.line([10, 10], [9, 41], line_width = 0.50, line_color="lightblue")
    p.line([17.6, 17.6], [9, 41], line_width = 0.50, line_color="lightblue")
    p.line([25.1, 25.1], [9, 41], line_width = 0.50, line_color="lightblue")
    p.line([32.6, 32.6], [9, 41], line_width = 0.50, line_color="lightblue")
    p.line([40, 40], [9, 41], line_width = 0.50, line_color="lightblue")
    p.line([9, 41], [10, 10], line_width = 0.50, line_color="lightblue")
    p.line([9, 41], [17.6, 17.6], line_width = 0.50, line_color="lightblue")
    p.line([9, 41], [25.1, 25.1], line_width = 0.50, line_color="lightblue")
    p.line([9, 41], [32.6, 32.6], line_width = 0.50, line_color="lightblue")
    p.line([9, 41], [40, 40], line_width = 0.50, line_color="lightblue")
    
    
    p.add_tools(HoverTool(renderers=[sp],
        tooltips=[('Sales Office',  '@Sales_Office'),
                  ('Label',  '@Legend')]))
    
    show(p)

box = interactive_output(show_this, {'Productivity':Productivity, 'Profitability':Profitability, 'timeline':timeline})


In [6]:
display(box, hbox)

Output()

<center>
    <img src=../Legend/Productivity_and_Profitability_Graphs.png  width="500000" height="250000" />
</center